In [1]:
!pip install langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.8 MB/s eta 0:00:00


In [2]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [3]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.0 MB/s eta 0:00:00


In [7]:
import re
import json
import io
import sys
from contextlib import redirect_stdout
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Set up Azure OpenAI API key and endpoint
AZURE_OPENAI_API_KEY = 'AZURE_OPENAI_API_KEY'  # Replace with your AZURE_OPENAI_API_KEY
AZURE_OPENAI_ENDPOINT = 'AZURE_OPENAI_ENDPOINT'  # Replace with your AZURE_OPENAI_ENDPOINT
OPENAI_API_VERSION = '2024-02-01'

model = AzureChatOpenAI(
    openai_api_version=OPENAI_API_VERSION,
    azure_deployment='gpt4',
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_API_KEY,
)

class GraphState(TypedDict):
    query_name: str
    data_file: str
    columns: str
    query: str
    program_code: str
    program_output: str
    error_message: str
    reflection: str
    execution_count: int
    terminate: bool

# Initialize the state with input data from files
def initialize(state):
    print("Executing Agent Initialize")
    # Read the initial input file
    with open('query_input.txt', 'r') as f:
        lines = f.readlines()
    state['query_name'] = lines[0].split(':')[1].strip()
    state['data_file'] = lines[1].split(':')[1].strip()
    state['columns'] = lines[2].split(':')[1].strip()

    # Read the query from the corresponding query file
    query_file = f"{state['query_name']}_query.txt"
    with open(query_file, 'r') as f:
        state['query'] = f.read().strip()

    state['execution_count'] = 0
    state['terminate'] = False
    return state

# Extract Python code block from the response content
def extract_code_from_response(response_content):
    code_block = re.search(r'```python(.*?)```', response_content, re.DOTALL)
    if code_block:
        return code_block.group(1).strip()
    return response_content.strip()

# Generate the initial Python program to query the CSV file
def gen_query_program(state):
    print("Executing Agent GenQueryProgram")
    prompt = f"""
    Write a Python program to perform the following query on a CSV file:
    {state['query']}
    The CSV file is named {state['data_file']} and contains the following columns: {state['columns']}.
    The program should output a JSON structure as specified in the query.
    """
    response = model.invoke([HumanMessage(content=prompt)])
    state['program_code'] = extract_code_from_response(response.content)
    return state

# Execute the generated Python program and capture its output
def execute_program(state):
    print(f"Executing Agent ExecuteProgram, count: {state['execution_count'] + 1}")
    state['execution_count'] += 1
    program_code = state['program_code']
    try:
        exec_globals = {}
        with io.StringIO() as buf, redirect_stdout(buf):
            exec(program_code, exec_globals)
            output = buf.getvalue()
        state['program_output'] = output.strip()
    except Exception as e:
        state['program_output'] = ""
        state['error_message'] = str(e)
    return state

# Check for errors in the program's output
def chk4r_err(state):
    print("Executing Agent Chk4rErr")
    if not state['program_output']:
        state['terminate'] = True
    else:
        try:
            json.loads(state['program_output'])
            state['terminate'] = False
        except json.JSONDecodeError:
            state['terminate'] = True
    return state

# Reflect on the errors and suggest a fix
def reflect_on_err(state):
    print("Executing Agent ReflectOnErr")
    reflection_prompt = f"""
    The following Python code failed to execute correctly:
    {state['program_code']}
    It produced the following error message: {state['error_message']}
    Reflect on why this error occurred and suggest a way to fix it.
    """
    reflection_response = model.invoke([HumanMessage(content=reflection_prompt)])
    state['reflection'] = reflection_response.content
    return state

# Regenerate the Python program based on the reflection
def regen_query_pgm(state):
    print("Executing Agent ReGenQueryPgm")
    regen_prompt = f"""
    Based on the following reflection, regenerate the Python program to fix the errors:
    {state['reflection']}
    Original prompt: Write a Python program to perform the following query on a CSV file:
    {state['query']}
    The CSV file is named {state['data_file']} and contains the following columns: {state['columns']}.
    The program should output a JSON structure as specified in the query.
    """
    regen_response = model.invoke([HumanMessage(content=regen_prompt)])
    state['program_code'] = extract_code_from_response(regen_response.content)
    return state

# Decide the next state based on the current execution state
def decide_next_state(state):
    if state['execution_count'] >= 3:
        state['terminate'] = True
        return END
    if state['terminate']:
        return "ReflectOnErr"
    return END

# Build the LangGraph workflow
workflow = StateGraph(GraphState)
workflow.add_node("Initialize", initialize)
workflow.add_node("GenQueryProgram", gen_query_program)
workflow.add_node("ExecuteProgram", execute_program)
workflow.add_node("Chk4rErr", chk4r_err)
workflow.add_node("ReflectOnErr", reflect_on_err)
workflow.add_node("ReGenQueryPgm", regen_query_pgm)

# Add edges between nodes
workflow.add_edge("Initialize", "GenQueryProgram")
workflow.add_edge("GenQueryProgram", "ExecuteProgram")
workflow.add_edge("ExecuteProgram", "Chk4rErr")
workflow.add_conditional_edges("Chk4rErr", decide_next_state, {"ReflectOnErr": "ReflectOnErr", END: END})

# Reflect and regenerate
workflow.add_edge("ReflectOnErr", "ReGenQueryPgm")
workflow.add_edge("ReGenQueryPgm", "ExecuteProgram")

# Set entry point
workflow.set_entry_point("Initialize")

# Compile and run the workflow
runnable = workflow.compile()
initial_state = GraphState(
    query_name="", data_file="", columns="", query="", program_code="",
    program_output="", error_message="", reflection="", execution_count=0, terminate=False
)

# Execute the workflow
result = runnable.invoke(initial_state)

# Handle final output
with open(f"{result['query_name']}.py", 'w') as f:
    f.write(result['program_code'])
with open(f"{result['query_name']}.txt", 'w') as f:
    f.write(result['program_output'])
with open(f"{result['query_name']}_errors.txt", 'w') as f:
    f.write(result['error_message'])
with open(f"{result['query_name']}_reflect.txt", 'w') as f:
    f.write(result['reflection'])

Executing Agent Initialize
Executing Agent GenQueryProgram
Executing Agent ExecuteProgram, count: 1
Executing Agent Chk4rErr
